In [59]:
from cassie import CassieEnv
import functions as f 
import loader as l 
import constants as c 
import numpy as np

import mujoco 

In [184]:
cassie = CassieEnv({})
cassie.reset()

(array([ 3.99322243e-01,  5.02738021e-01,  3.85149473e-01,  8.79052870e-01,
         1.27200696e+00,  4.99900239e-01, -4.08194630e-01,  1.27271287e+00,
         5.99526691e-01,  4.97174572e-01,  3.85678299e-01,  1.19310330e+00,
         1.27142680e+00,  4.99942236e-01, -4.08265191e-01,  1.27270126e+00,
         4.99953514e-01,  4.99961098e-01,  5.00137610e-01,  9.99970236e-01,
        -4.30786895e-06,  7.70190819e-03,  4.56580093e-04,  0.00000000e+00,
         1.00000000e+00]),
 {})

In [202]:
cassie.step(cassie.action_space.sample())



(array([ 0.18744962,  0.38710523,  0.28759113,  0.94130226,  0.77137132,
         0.49880466, -0.40128582,  1.26270015,  0.28461942,  0.27005856,
         0.30519476,  1.06042893,  1.05480874,  0.50081429, -0.40008793,
         1.2683689 ,  0.50214241,  0.48617227,  0.5285493 ,  0.89367805,
        -0.00889322, -0.43935034,  0.09072884,  0.9781476 , -0.20791169]),
 0.5950386202251442,
 False,
 False,
 {})

In [203]:
import tensorboard

In [204]:
import glfw
from pyvirtualdisplay import Display

# Start virtual display
display = Display(visible=0, size=(800, 600))
display.start()

# Initialize GLFW library
if not glfw.init():
    raise Exception("GLFW initialization failed")



In [205]:
cassie = CassieEnv({})


In [207]:
#simulate cassie for some steps doing nothing and plot the metrics inside 
rendered_frames = []
used_quantities = []
cassie.render_mode = "rgb_array"
cassie.reset()
for _ in range(100):
    obs, _, _, _,_ = cassie.step(np.zeros(10))
    rendered_frame = cassie.render()
    rendered_frames.append(rendered_frame)
    for i in range (len(obs)):
        cassie.used_quantities["ob_"+str(i)] = obs[i]
    used_quantities.append(cassie.used_quantities)

# convert list of dictionaries to dictionary of lists
used_quantities = {k: [d[k] for d in used_quantities] for k in used_quantities[0]}

In [208]:
import matplotlib.pyplot as plt
import ipywidgets as widgets

def plot_cassie(rendered_frames, used_quantities):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))
    fig.subplots_adjust(bottom=0.2)

    # plot the first frame
    im = ax1.imshow(rendered_frames[0])
    ax1.axis('off')

    # plot the metrics
    lines = {}
    for key, value in used_quantities.items():
        line, = ax2.plot(value[:1])
        lines[key] = line
    ax2.set_xlabel('Frame')
    ax2.set_ylabel('Value')

    # create the slider
    slider = widgets.IntSlider(min=0, max=len(rendered_frames)-1, step=1, value=0)

    # create an output widget to display the plot
    out = widgets.Output()

    # function to update the plot when the slider is moved
    def update_plot(frame):
        im.set_data(rendered_frames[frame])
        for key, line in lines.items():
            line.set_data(range(frame+1), used_quantities[key][:frame+1])
        ax2.relim()
        ax2.autoscale_view()
        with out:
            out.clear_output(wait=True)
            display(fig)

    slider.observe(lambda event: update_plot(event['new']), names='value')

    return widgets.VBox([out, slider])


In [210]:
rendered_frames[0].shape

(480, 480, 3)

In [212]:
import cv2
import numpy as np

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # mp4 codec
out = cv2.VideoWriter('output.mp4', fourcc, 25.0, (480, 480)) # output file name, codec, frame rate, frame size

# Loop through the list of frames and write each frame to the output video file
for frame in rendered_frames:
    # Resize the frame to match the output frame size
    frame = cv2.resize(frame, (480, 480))
    out.write(frame)

# Release the VideoWriter and close all windows
out.release()
cv2.destroyAllWindows()
